# Machine Learning Application: Sustainable Biofuels

---

### Goals of the notebook

* Train a machine learning model to predict the flash point of fuels using their spectra

* Apply the train test split technique to validate our machine learning model

* Use feature selection as a way to avoid the machine learning model from overfitting


---

### Table of Contents

* [Import spectra and flash point data + EDA](#section1)<br>

* [EXERCISE 1 - Normalization](#section2)<br>

* [Train Test Split](#section3)<br>

  * [Overfitting](#subsection1)<br>

* [EXERCISE 2 - Feature Selection](#section4)<br>

  * [Task A](#subsection2)<br>

  * [Task B](#subsection3)<br>

* [EXERCISE 3 - Model Training](#section5)<br>

  * [Linear Regression](#subsection4)<br>

  * [Random Forest Regressor](#subsection5)<br>

* [EXERCISE 4 - Challenge](#section6)<br>


---

Currently, aviation accounts for a big part of Greenhouse Gas (GHG) emissions, and switching to sustainable aviation fuel could halve carbon emissions by 2050.

However, the development of sustainable aviation fuels is limited by significant technical barriers. Specifically, because property testing for novel fuels is expensive and requires a large volume of the sample, it is performed pretty late in the development cycle. This leads to the scale-up of blends that are ultimately not viable to be used as fuels or underperform.

The purpose of this exercise is to train a machine learning model capable of predicting fuel properties early in the development cycle. This can be used as a first pass check before scale up of promising blends, helping derisk and accelerate alternative jet fuel research and development.



Let's first get started by importing any libraries that are needed.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import spectra and flash point data + EDA <a name = "section1">

Using pandas, let's import the 'flash_point_and_spectra_data_short.csv' database from the data folder. Then we will perform some EDA to get to know the dataset.

In [ ]:
df=pd....

In [ ]:
#Exploratory Data Analysis 1

#This can be head, tail, columns, describe, etc.

df...

In [ ]:
#Exploratory Data Analysis 2

...

In [ ]:
#Exploratory Data Analysis 3

...

This database has the columns 'fuel', and 'flash point', which give you the flash point and names of different fuels.  

The other columns, represent the FTIR spectra of these fuels. FTIR spectra is like the fingerprint of a molecule or blends of molecules, it gives us insights into the structure of the sample. The data is taken at the frequencies indicated by the column names of the database.


### Tasks:

In the cells below, find out:

- How many samples does this database have? *Hint: What function that we have used before that lets us find the number of rows of a DataFrame?*
- How many spectra values do we have for each sample?

In [ ]:
# EXERCISE - How many samples does the database have?

...

In [ ]:
# EXAMPLE - How many spectra values do we have for each sample?

# We could use a for loop like this:
spectra_columns=[]
for column in df.columns:
  if ('flash point' not in column) and ('fuel' not in column):
    spectra_columns.append(column)
len(spectra_columns)

# or we could use list comprehensions.
# The syntax for a list comprehension is: [expression(element) for element in old_list if condition]
# For example, the for loop above could be replaced by:
len([c for c in df.columns if 'flash point' not in c and 'fuel' not in c])

Let's see what an FTIR spectra looks like.

Plot the spectra of the first fuel:
- the x-axis will be the columns of the database, excluding the 'flash point' and 'fuel' columns. These are the frequencies the spectra data is taken at.
- the y-axis will be the values of those columns in the first row (again, excluding the 'fuel' and 'flash point' columns). This is the absorbance of a specific sample and the frequencies in the x-axis.


In [ ]:
x_data = [pd.to_numeric(c) for c in df.columns if 'fuel' not in c and 'flash point' not in c]
y_data = df.loc[0,[c for c in df.columns if 'fuel' not in c and 'flash point' not in c]]

# Go back to your previous notebook 08 data visualization to remind yourself how to use matplotlib to create axes labels and create a line plot.

plt.title('...')
plt.xlabel('...')
plt.ylabel('...')
plt.plot(x_data, y_data)

We will be using the spectra curves to predict the flash point of these fuels.

# EXERCISE 1- Normalization <a name = "section2">

Due to intrumental error and noise, the raw spectra are not always comparable between each other. For example, taking the spectra of the same sample in two different rooms with different moisture levels can result in different spectra.

To help mitigate some of this error, we will be normalizing the spectra.

Normalizing data means that we adjust the scale of our data by comparing our numbers on a relative scale instead of directly comparing them. This helps with data that takes on a large range of numbers, and it eliminates the issue of comparing categories that are on different scales.


### Tasks:

* 1a. Replace any negative values with 0.

*For Task 1a, look up (on the internet) the .clip() function from pandas.*

* 1b. Normalize the spectra of each sample so that the maximum value for each spectra is 1.

In [ ]:
# EXERCISE 1a.
all_spectra_data=df[[c for c in df.columns if 'fuel' not in c and 'flash point' not in c]]

# Use the pandas function .clip() to clip any negative values from all_spectra_data
all_spectra_data_clipped=all_spectra_data.clip(lower=...)


In [ ]:
# EXERCISE 1b.
#The following line gives you the maximum value for each row of spectra
maximum_spectra_value_for_each_row= all_spectra_data_clipped.max(axis=1)

#Replace '...' in the following line to divide all_spectra_data by the maximum spectra value for each row
all_spectra_data_clipped_and_normalized=all_spectra_data_clipped.div(...,axis=0) #rename variable

# Train Test Split <a name="section3">

We will use the spectra **(features)** to predict the flash point **(target variable)** of these fuels.

## Splitting into train and testing
Although we want to use all our data to build a model, it is important to also get a sense of how the model will be able to predict new data. Because we can't magically generate new data, we will be setting a portion of our data aside (called the **test set**) to test our models. Usually, 20% to 30% of the data is used as a test set.

The rest of the data is called a **train set**, and we will be using it to train our models. After training a model, we see how well the model predicts the property in the test set.

### Task: split data into training and testing with the help of sklearn's train_test_split
You will reserve 30% of the data for testing, set a random seed of 42 to ensure reproducibility. All "ensure reproducibility" means in this case is that you will get the same train test split of your data every time you run the cell. For example, if I didn't have an argument defining my random state, then if I have to rerun my notebook or rerun the cell, I would get different data in my training vs test set which I don't want to happen. I want the data in my training and test set to be consistent throughout my notebook.

The number 42 has no inherent significance (unless you have read The Hitchiker's Guide to the Galaxy), but is often used as a random seed for simplicity.


## Each variable is explained below:
* X_train: The spectra of fuels in the train set
* X_test: The spectra of fuels in the test set
* y_train: The flash point of fuels in the train set
* y_test: The flash point of fuels in the test set

In [ ]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test=train_test_split(all_spectra_data_clipped_and_normalized,
                                              df['flash point'],test_size=0.3,random_state=42)

## Overfitting<a name="subsection1">

When we have too many features, choosing which ones to include or exclude is one of the most important parts of training a model. We could use all the spectra values to train a model that would fit the training data perfectly, but this will result in overfitting.

Overfitting happens when the model follows the training data too closely, and picks up any variability or noise from the data. In this case, the model will be able to predict the training set really well but will fail to predict new data accurately.

<img src="https://drive.google.com/uc?export=view&id=1CYtxRiUkks0_4ZI32MG2ZG_kGg-CXeyc" width="500" height="200" />

To help illustrate this, consider the following example:
- Imagine a student is studying for a math test by practicing with a set of problems.
- Instead of learning the general method to solve a problem, they memorize the answers to each specific question.
- The day of the test, the problems are similar but not exactly the same as the practice problems.
- Because they memorized the specific answers, rather than understanding how to solve the problems, they struggle to answer the questions on the test.

When this happens in machine learning, we may get a model that is only able to accurately predict samples in the training set. That model would not be useful.


To help avoid overfitting, we must cut down on the number of features and only select the ones that will help us predict the flash point accurately, this process is called **feature selection**.


### Task:
Fit a model with all variables to your training set.

Similar to the Cell Phone Design Challenge, we will need to specify a machine learning model to help us with **predicting** flash point of fuels using spectra.

We will again be using the `scikit-learn` library which implements a variety of different machine learning models and other analysis tools. As we are dealing with **regression** in machine learning, we will specifically be using the **Random Forest Regressor** model. (You can learn more about random forest models from the [scikit-learn user guide](https://scikit-learn.org/stable/modules/ensemble.html#forest)).

In [ ]:
#Note: this may take a while

from sklearn.ensemble import RandomForestRegressor
sample_model=RandomForestRegressor(random_state=42)
sample_model.fit(X_train,y_train)


There are different ways of calculating the performance of a model. In the Cell Phone Design Challenge, you used the root mean squared error (RMSE). For this exercise, you will use a different error metric called mean absolute error (MAE). There are a couple of differences between these two metrics:
- RMSE gives more weight to larger errors (because of the squaring step), while MAE is not affected by them as much.
- MAE is easier to interpret, it is simply the average error for all the data.

Both are useful to measure accuracy, but highlight different aspects of the errors in our predictions.

To calculate the mean absolute error, you will be using the scikit-learn's `mean_absolute_error()` function.

In [ ]:
from sklearn.metrics import mean_absolute_error

print('Mean Absolute Error: '+str(mean_absolute_error(y_test,sample_model.predict(X_test))))

**We will be comparing this MAE to that of a model trained after feature selection.**

# Exercise 2: Feature Selection <a name = "section4">

Recall that we use the process of **feature selection** to avoid overfitting. We must must cut down on the number of features (in our case spectra) and only select the ones that will help us predict the flash point accurately.

## Task A: <a name = "subsection2">

Some of the wavelengths have very low values for most our samples, these could represent functional groups (an atom or group of atoms within a molecule that has similar chemical properties) and chemical bonds that are not present in any fuel.

In the cell below, delete the columns that have a value of less than 0.1 for at least 50% of all samples.

In [ ]:
# Task A - fill in the '...'

# Use the variance_threshold function
from biofuel_helper_functions import variance_threshold

columns_less_than_threshold=variance_threshold(X_train,minimum_value=...,sample_percentage=...)


X_train=X_train.drop(columns_less_than_threshold,axis=1)
X_test=X_test.drop(columns_less_than_threshold,axis=1)


Let's find the mean absolute error again. How does the accuracy change? Should we keep these columns? Should we delete more?

In [ ]:
# Fill in the '...' (see how mean_absolute_error was used a couple cells above)

threshold_model=RandomForestRegressor(random_state=42)
threshold_model.fit(X_train,y_train)
print('Mean Absolute Error: '+str(mean_absolute_error(...,threshold_model.predict(...))))

## Task B: <a name = "subsection3">
We will be introducing two types of feature selection: binning and recursive feature elimination

### Task B1
- The binning function will aggregate multiple columns together. For example, if we had values at 2500,2501,2502,2503,2504,and 2505 cm-1 before, we could bin all these values into 1 column by averaging them. It's a way of smoothing the spectra of these fuels to eliminate tiny peaks that may not be important. The parameters to change are the "width" of the window we are smoothing by.


For example, in the image below, the window width is 1, and we are averaging 3 values into the same bin.

<img src="https://drive.google.com/uc?export=view&id=1MmAVqwWPxeP6RBLT1ovLsuVIcC_ZIpJu" width="900" height="450" />



### Task B2
- Recursive feature elimination is a type of feature selection method that searches for a subset of features by removing the least important ones. It works by fitting a model, ranking the features based on importance, and removing the X least important ones. Then, it fits another model with the remaining features and repeats this process until there are Y features left. The ranking of these features is the reverse order in which they were eliminated, i.e.: the ones left at the very end are ranked 1 and seen as most important, while the first ones to be eliminated are ranked last and see as the least important features. There are different parameters to change here, including how many features to delete at each step (the X), and when to stop the process (when we have reached Y features).

<img src="https://drive.google.com/uc?export=view&id=1y2zfLA0SYP-ml37cea_4Tt2HKy7E84ka" width="1000" height="600" />

## Goal: Choose a feature selection method and reduce the number of features.


Use the bin_spectra_features function to bin spectra with a window width of 40, then see the error on the test set.

In [ ]:
# Task B1

from biofuel_helper_functions import bin_spectra_features

bin_model=RandomForestRegressor(random_state=42)
bin_X_train=bin_spectra_features(X_train,window_width=...)
bin_X_test=bin_spectra_features(...,window_width=...)
bin_model.fit(bin_X_train,y_train)
print('Mean Absolute Error: '+str(mean_absolute_error(y_test,bin_model.predict(...))))


Now use recursive feature elimination with a step size of 100 to select the 50 most important features. Your n_features_to_select is when you want to stop the process of recursive feature elimination (in this case when you have the 50 most important features). Your step is how many features you want to eliminate at each "elimination round."

In [ ]:
# Task B2
#RFE: https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html
## Use RandomForestRegressor as the estimator
## This process may take a while depending on step size and final number of features chosen
## trying n_features_to_select=50 and step=20 will take about a minute to run, any step size smaller will take longer

from sklearn.feature_selection import RFE
import time
tic=time.time()
rfe=RFE(estimator=RandomForestRegressor(random_state=42),n_features_to_select=...,step=...)



rfe.fit(X_train,y_train)
rfe_X_train=rfe.transform(X_train)
rfe_X_test=rfe.transform(X_test)
rfe_model=RandomForestRegressor(random_state=42)
rfe_model.fit(rfe_X_train,y_train)
toc=time.time()

print('Mean Absolute Error: '+str(mean_absolute_error(... , ...)))

Let's plot what our predictions with recursive feature elimination look like to get a visual sense of model performance.

In [ ]:
train_predictions=rfe_model.predict(rfe_X_train)
test_predictions=rfe_model.predict(rfe_X_test)

plt.plot(np.arange(260,310,1),np.arange(260,310,1),c='grey')
plt.scatter(train_predictions,y_train,label='Train set')
plt.scatter(test_predictions,y_test,label='Test set')
plt.legend()

If you have time, train multiple models with different step sizes, the plot the accuracy.


In [ ]:
# try a step size of 75,50,and 25 and plot their accuracy against the step size 0f 100

#first, initialize the RFE models
rfe_step75=RFE(estimator=RandomForestRegressor(random_state=42),n_features_to_select=50,step=...)
rfe_step50=RFE(estimator=RandomForestRegressor(random_state=42),n_features_to_select=50,step=...)
rfe_step25=RFE(estimator=RandomForestRegressor(random_state=42),n_features_to_select=50,step=...)




#then fit the RFE models
rfe_step75.fit(X_train,y_train)
rfe_step50.fit(X_train,y_train)
rfe_step25.fit(X_train,y_train)

#transform the train and test sets
rfe_X_train75=rfe_step75.transform(X_train)
rfe_X_test75=rfe_step75.transform(X_test)
rfe_X_train50=rfe_step50.transform(X_train)
rfe_X_test50=rfe_step50.transform(X_test)
rfe_X_train25=rfe_step25.transform(X_train)
rfe_X_test25=rfe_step25.transform(X_test)


#train random forest models
rfe_model_step75=RandomForestRegressor(random_state=42)
rfe_model_step75.fit(rfe_X_train75,y_train)
rfe_model_step50=RandomForestRegressor(random_state=42)
rfe_model_step50.fit(rfe_X_train50,y_train)
rfe_model_step25=RandomForestRegressor(random_state=42)
rfe_model_step25.fit(rfe_X_train25,y_train)

#get the train and test predictions for each model
train_predictions_75=rfe_model_step75.predict(rfe_X_train75)
train_predictions_50=rfe_model_step50.predict(rfe_X_train50)
train_predictions_25=rfe_model_step25.predict(rfe_X_train25)
test_predictions_75=rfe_model_step75.predict(rfe_X_test75)
test_predictions_50=rfe_model_step50.predict(rfe_X_test50)
test_predictions_25=rfe_model_step25.predict(rfe_X_test25)

#plot the train and test mean absolute error
plt.scatter([100,75,50,25],[mean_absolute_error(train_preds,y_train) for train_preds in [train_predictions,train_predictions_75,train_predictions_50,train_predictions_25]],c='tab:blue',s=10,label='train mean absolute error')
plt.plot([100,75,50,25],[mean_absolute_error(train_preds,y_train) for train_preds in [train_predictions,train_predictions_75,train_predictions_50,train_predictions_25]],c='tab:blue')

plt.scatter([100,75,50,25],[mean_absolute_error(test_preds,y_test) for test_preds in [test_predictions,test_predictions_75,test_predictions_50,test_predictions_25]],c='tab:orange',s=10,label='test mean absolute error')
plt.plot([100,75,50,25],[mean_absolute_error(test_preds,y_test) for test_preds in [test_predictions,test_predictions_75,test_predictions_50,test_predictions_25]],c='tab:orange')

plt.xlabel('RFE step size')
plt.ylabel('Mean Absolute error')
plt.legend()



# Exercise 3: Model Training <a name = "section5">
We will be testing the features we found with two different models: **Linear Regression** and a **Random Forest Regressor**. We will walk through the steps through Linear Regression and you will get to try coding the Random Forest Regressor!

## Task:
Train both models and see how well it predicts both the training set and the test set. We will use the following to evaluate our models:

- Mean absolute error

- Parity plot showing predicted flash point vs. experimental flash point for all the data

## Linear Regression <a name = "subsection4">

First, **specify the model**:

In [ ]:
from sklearn.linear_model import LinearRegression

linreg=LinearRegression()

Now we will **train the model**.

We will use `rfe_X_train` as our input features (spectra) and `y_train` as our target variable (flash point).

In [ ]:
linreg.fit(rfe_X_train,y_train)

Let's **evaluate our model** by finding the mean absolute error and creating a parity plot.

In [ ]:
print('Mean Absolute Error for Linear Regression: '+str(mean_absolute_error(y_test,linreg.predict(rfe_X_test))))

train_predictions=linreg.predict(rfe_X_train)
test_predictions=linreg.predict(rfe_X_test)
plt.plot(np.arange(260,310,1),np.arange(260,310,1),c='grey')
plt.scatter(...,y_train,label='Train set')
plt.scatter(...,y_test,label='Test set')
plt.legend()

## Random Forest Regressor <a name = "subsection5">

Now your turn! First let's **specify the model**.

Make sure to use the argument `random_state=42`

*Hint: Look up the documentation on [Random Forest Regressor models on sci-kit learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) to help you.*

In [ ]:
from ... import ...
rf=...


Now we will **train the model**.

Similar to Linear Regression, we will use `rfe_X_train` as our input features (spectra) and `y_train` as our target variable (flash point).

In [ ]:
rf.fit(..., ...)


Let's **evaluate our model** by finding the mean absolute error and creating a parity plot.

In [ ]:
print('Mean Absolute Error for RandomForest: '+str(mean_absolute_error(... , ...)))

train_predictions=rf.predict(...)
test_predictions=rf.predict(...)
plt.plot(np.arange(260,310,1),np.arange(260,310,1),c='grey')
plt.scatter(...,y_train,label='Train set')
plt.scatter(...,y_test,label='Test set')
plt.legend()

## DISCUSSION

Which model worked better? Why? Support your reasoning with evidence from evaluating the models.

# Exercise 4 - Challenge <a name = "section6">
Your friend just made this fuel that they're convinced is the perfect fuel. Your challenge is to predict the flash point for this new fuel with your model.

Do not forget to do the same transformations (normalizing, deleting the same features) as with your training!

Note that the new database does not have the 'fuel' and 'flash point' columns.

In [ ]:
new_spectra=pd.read_csv('data/jet_a_spectra_short.csv')

new_spectra.head()


First, clip the negative values

In [ ]:
#clip negative values
new_spectra=new_spectra.clip(lower=...)


Then, normalize the data

In [ ]:
#normalize the new data
maximum_spectra_value_for_new_spectra=new_spectra.'...'
new_spectra=new__spectra.div(...)


In [ ]:
# delete the columns that didn't pass the variance threshold from before with the .drop() function.
# for this, we will simply keep the columns that were kept in X_train
columns_less_than_threshold=[c for c in new_spectra.columns if c not in X_train.columns]
new_spectra=new_spectra.drop(...,axis=1)


We will now pick a feature selection method and model that we like.
You will need to transform the new data the same way that you did the data from yesterday.

For example, if you want to use a model that was trained on binned data, don't forget to also bin the new data.

In [ ]:
## pick the feature selection method of your choice, you could bin the data, use recursive feature elimination, or both.

In [ ]:
#if you want to use binning:
from ... import ...
new_spectra_binned=bin_spectra_features(...,window_width=...)
bin_X_train=bin_spectra_features(...,window_width=...)
bin_X_test=bin_spectra_features(...,window_width=...)


# if you want to use recursive feature elimination
from ... import ...
rfe=RFE(estimator=RandomForestRegressor(random_state=42),n_features_to_select=...,step=...)
rfe.fit(... , ...)
rfe_X_train=rfe.transform(...)
rfe_X_test=rfe.transform(...)
new_spectra_rfe=rfe.transform(...)

Now pick a model, train it to the data from yesterday (pick the training set using the same feature selection method as the one you chose above), and predict the flash point of the new fuel

In [ ]:
#import the model of your choice
from ... import ...
model= ...

### Here are your choices for the above: ###
## Random Forest Regressor
#from sklearn.ensemble import RandomForestRegressor
#model = RandomForestRegressor(random_state=42)
# OR
## Linear Regression
#from sklearn.linear_model import LinearRegression
#model=LinearRegression()


# fit the model to the appropriate train set (either binned or with RFE)
model.fit(... , ...)


# then predict the new spectra.
# if you trained your model with binned train data, use the binned new spectra from the cell above
# if you trained your model with recursive feature elimination as a feature selection method, use the new spectra that was also transformed with recursive feature elimination
new_prediction=model.predict(...)

print(new_prediction)


They were able to measure the flash point for this fuel at 318.15K, how close is your model to the actual value? \
Let's plot the predictions for the test set and the new fuel below.

In [ ]:
# fill in the '...' to match the version of the train and test set (binned or RFE) that you used above
train_predictions= model.predict(...)
test_predictions=model.predict(...)


plt.plot(np.arange(260,310,1),np.arange(260,310,1),c='grey')
plt.scatter(train_predictions,y_train,label='Train set')
plt.scatter(test_predictions,y_test,label='Test set')
plt.scatter(new_prediction,315.15,label='New fuel')
plt.legend()

Repeat Exercises 2 and 3 in this notebook above, changing different parameters for your model and feature selection, did the accuracy increase?

# Summary:

What worked best for the combination of feature selection and model? What went wrong? What can we do differently to increase the accuracy of our model?

Notebook developed by: Ana Comesana